In [33]:
import pandas as pd 

df = pd.read_csv('VIP_모델학습용_8902_4940.csv')

/var/folders/3p/8hc9c44s38j902ytq253kt6m0000gn/T/ipykernel_4351/530037132.py:3: DtypeWarning: Columns (3323,3324,3325,3830,3831,3832,3833,3834,3835) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('VIP_모델학습용_8902_4940.csv')


In [34]:
exclude_cols = [
    'Score_BadDebt', 'Score_Delinq', 'Score_Activity', 'Score_Asset', 
    'Score_Status_Total', 'Slope_Spend', 'Slope_Balance', 'Slope_Count', 
    'Norm_Slope_Spend', 'Norm_Slope_Balance', 'Norm_Slope_Count', 
    'Score_Slope_Total', 'Final_Total_Score', 'Score_Clipped', 
    'Risk_Score_Centered', 'cluster_label', 'cluster_label_swapped', 
    'y_jitter', 'Target', '발급회원번호' 
]

In [35]:
X = df.drop(columns=[col for col in exclude_cols if col in df.columns])
y = df['Target']

In [36]:
from sklearn.preprocessing import LabelEncoder

# 1. 문자열(object) 컬럼들만 쏙쏙 찾아내기
cat_cols = X.select_dtypes(include=['object']).columns

# 2. LabelEncoder로 숫자로 바꾸기
# 상림아, 결측치(NaN)가 있으면 에러가 날 수 있으니 문자열 "None"으로 채우고 시작할게!
le = LabelEncoder()

for col in cat_cols:
    # NaN 값을 'Unknown'이라는 문자로 채워주고 숫자로 변환!
    X[col] = X[col].fillna('Unknown').astype(str)
    X[col] = le.fit_transform(X[col])

In [41]:
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. 수치형 컬럼 빈칸 0으로 채우기 (SHAP/VIF 에러 방지!)
X = X.fillna(0)

# 2. 학습용(Train)과 테스트용(Test) 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. 데이터 불균형을 고려한 가중치 계산
ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

# 4. XGBoost 모델 생성 및 학습
model_full = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    scale_pos_weight=ratio, # 이탈자에게 용기를 팡팡!
    random_state=42,
    eval_metric='logloss'
)

model_full.fit(X_train, y_train)

print("상림아! 전체 피처를 사용한 모델 학습이 완료됐어! 🌈🐟")

상림아! 전체 피처를 사용한 모델 학습이 완료됐어! 🌈🐟


In [42]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 테스트 데이터(X_test)로 예측하기
y_pred = model_full.predict(X_test)

# 2. 전체 정확도 계산
accuracy = accuracy_score(y_test, y_pred)

print(f"상림아! 전체 피처 모델의 정확도는 {accuracy:.4f}이야! 🌈")
print("\n--- 📋 상세 분류 성적표 (Classification Report) ---")
# '1' 행의 recall(재현율) 수치가 이탈자를 잡아낸 비율이야!
print(classification_report(y_test, y_pred))

# # 3. 혼동 행렬(Confusion Matrix) 시각화
# # 실제로 맞춘 것과 틀린 것을 한눈에 보자!
# cm = confusion_matrix(y_test, y_pred)
# plt.figure(figsize=(6, 5))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

상림아! 전체 피처 모델의 정확도는 0.9444이야! 🌈

--- 📋 상세 분류 성적표 (Classification Report) ---
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1491
           1       0.86      0.79      0.82       290

    accuracy                           0.94      1781
   macro avg       0.91      0.88      0.89      1781
weighted avg       0.94      0.94      0.94      1781



In [44]:
import pandas as pd
import numpy as np
import shap
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. SHAP Value 계산 (전체 피처 대상)
# 모델이 어떤 변수에 반응했는지 전수 조사를 시작할게!
explainer = shap.TreeExplainer(model_full)
shap_values = explainer.shap_values(X_test)

# 변수별 평균 절대 SHAP 값을 계산해서 중요도 데이터프레임 만들기
shap_importance = np.abs(shap_values).mean(axis=0)
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'SHAP_Importance': shap_importance
}).sort_values(by='SHAP_Importance', ascending=False)

# 2. 상위 중요 변수들에 대한 VIF 계산
# 피처가 너무 많으면 VIF 계산이 멈출 수 있어. 
# 우선 가장 중요한 상위 50개 피처를 골라서 VIF를 측정해줄게!
top_n = 50
top_features = importance_df.head(top_n)['Feature'].tolist()
X_vif_target = X[top_features]

print(f"상림아! 상위 {top_n}개 핵심 피처의 VIF 지수를 계산 중이야. 조금만 기다려줘! 🌈")

vif_data = pd.DataFrame()
vif_data["Feature"] = top_features
vif_data["VIF"] = [variance_inflation_factor(X_vif_target.values, i) for i in range(len(top_features))]

# 3. SHAP와 VIF 합치기
# 두 정보를 하나로 합쳐서 상림이가 보기 편하게 정렬할게!
final_analysis = pd.merge(importance_df, vif_data, on='Feature', how='left')

# VIF가 결측치인 건 상위 50위 밖의 변수들이야!
print(final_analysis.head(20))

# CSV 파일로 저장해서 나중에 보고서에 바로 쓰자!
final_analysis.to_csv('VIP_WiderFormat_Shap_Vif.csv', index=False, encoding='utf-8-sig')

상림아! 상위 50개 핵심 피처의 VIF 지수를 계산 중이야. 조금만 기다려줘! 🌈


/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


            Feature  SHAP_Importance           VIF
0   이용건수_일시불_B0M_12         1.240356  1.098850e+05
1    이용건수_신용_B0M_12         0.710747  2.762844e+05
2    이용건수_신용_R3M_12         0.628235  1.080745e+06
3   이용건수_일시불_R3M_12         0.524226  4.946697e+05
4    이용건수_신용_R3M_09         0.420482  1.174807e+06
5   이용건수_일시불_R3M_09         0.382424           inf
6    이용건수_신판_R3M_12         0.371672  1.546973e+06
7    이용건수_신판_B0M_07         0.353547  3.962352e+05
8    이용건수_신용_B0M_07         0.338043  3.015606e+05
9   이용건수_일시불_B0M_11         0.328548  1.415565e+05
10   이용건수_신판_R3M_09         0.321982  1.609037e+06
11   이용건수_신용_B0M_11         0.265375  3.032559e+05
12   이용금액_신판_B0M_07         0.238251  4.741130e+02
13  이용건수_일시불_B0M_07         0.230263           inf
14  이용금액_일시불_B0M_12         0.206168  2.214881e+02
15   이용건수_신판_B0M_12         0.200741  3.773834e+05
16   이용금액_신판_B0M_12         0.188202  6.775198e+02
17   이용건수_신판_B0M_11         0.176811  4.332760e+05
18    _1순위카드이용건수_12         0.1